In [9]:
library(dplyr)
library(lubridate)
library(tidyr)

#### <font color = yellow> Get transactions (Buy/Sell) for seperate stocks (saved from Excel table to the generic file, stock_input.csv )


In [2]:
df <- read.csv(file = "stock_input.csv")
dim(df)
tail(df,2)

[1] 57  6

,Date,Type,Stock,Quantity,Price,Disburse
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>
56,12/8/2023,Sell,AI,-200,27.403,5480.46
57,12/12/2023,Sell,AI,-100,27.371,2737.07


### <font color = yellow> Remove unwanted last row from the new df - use only when last row has NA's

In [3]:
# last_row <- nrow(df)

# df <- df[-nrow(df), ]
# dim(df)

### <font color = yellow> Format the the Date, data type (not used in this notebook)

In [4]:
# Convert Date column to Date format 
df$Date <- as.Date(df$Date, format = "%m/%d/%Y")
tail(df)

,Date,Type,Stock,Quantity,Price,Disburse
,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>
52,2023-09-28,Buy,AI,200,24.020,-4804.00
53,2023-09-28,Buy,AI,400,24.908,-9963.20
54,2023-11-20,Sell,AI,-300,31.010,9302.88
55,2023-12-08,Sell,AI,-200,27.508,5501.46
56,2023-12-08,Sell,AI,-200,27.403,5480.46
57,2023-12-12,Sell,AI,-100,27.371,2737.07


#### <font color = yellow> Process: create transactions - matches each Sell (quan) with one or more Buy(s) using FIFO

In [5]:
# Splitting buys and sells
buys <- subset(df, Type == "Buy")
sells <- subset(df, Type == "Sell")

# Sorting buys and sells by date
buys <- buys[order(buys$Date),]
sells <- sells[order(sells$Date),]

# LIFO Matching Process with combined buys for each sell
transaction_number <- 1
transaction_results <- list()
total_proceeds <- 0
total_costs <- 0
total_shares_sold <- 0
total_shares_bought <- 0
total_pnl <- 0
stock_id <- df$Stock[1]

for (i in 1:nrow(sells)) {
  sell_row <- sells[i,]
  remaining_qty <- abs(sell_row$Quantity)
  
  combined_buys <- data.frame(BuyDate = as.Date(character()), BuyQty = integer(), BuyPrice = double())
  pnl <- 0  # Initialize profit/loss
  proceeds <- 0  # Initialize proceeds from sales
  cost <- 0  # Initialize cost of purchases
  
  while (remaining_qty > 0 && nrow(buys) > 0) {
    # Select the most recent buy
    buy_row <- buys[1,]
    
    if (buy_row$Date <= sell_row$Date) {  # Ensure buy date is before sell date
      if (remaining_qty >= buy_row$Quantity) {
        pnl <- pnl + (sell_row$Price * buy_row$Quantity) - (buy_row$Price * buy_row$Quantity)
        proceeds <- proceeds + (sell_row$Price * buy_row$Quantity)
        cost <- cost + (buy_row$Price * buy_row$Quantity)
        combined_buys <- rbind(combined_buys, data.frame(BuyDate = buy_row$Date, BuyQty = buy_row$Quantity, BuyPrice = buy_row$Price))
        remaining_qty <- remaining_qty - buy_row$Quantity
        buys <- buys[-1,]  # Remove the matched buy
      } else {
        pnl <- pnl + (sell_row$Price * remaining_qty) - (buy_row$Price * remaining_qty)
        proceeds <- proceeds + (sell_row$Price * remaining_qty)
        cost <- cost + (buy_row$Price * remaining_qty)
        combined_buys <- rbind(combined_buys, data.frame(BuyDate = buy_row$Date, BuyQty = remaining_qty, BuyPrice = buy_row$Price))
        buys[1, "Quantity"] <- buy_row$Quantity - remaining_qty
        remaining_qty <- 0
      }
    } else {
      break  # If no more buys before the sell date, exit the loop
    }
  }
  
  total_proceeds <- total_proceeds + proceeds
  total_costs <- total_costs + cost
  total_shares_sold <- total_shares_sold + abs(sell_row$Quantity)
  total_shares_bought <- total_shares_bought + sum(combined_buys$BuyQty)
  total_pnl <- total_pnl + pnl

  # Store transaction details
  transaction_results[[transaction_number]] <- list(
    SellDate = sell_row$Date, SellQty = sell_row$Quantity, SellPrice = sell_row$Price,
    Buys = combined_buys, PnL = pnl
  )
  
  transaction_number <- transaction_number + 1
}

# Printing the result
for (k in 1:length(transaction_results)) {
  cat(sprintf("Transaction %d\n", k))
  cat("Sell\n")
  cat(sprintf("%s  %d  %.3f\n", transaction_results[[k]]$SellDate, transaction_results[[k]]$SellQty, transaction_results[[k]]$SellPrice))
  cat("Buys\n")
  for (j in 1:nrow(transaction_results[[k]]$Buys)) {
    cat(sprintf("%s  %d  %.3f\n", transaction_results[[k]]$Buys$BuyDate[j], transaction_results[[k]]$Buys$BuyQty[j], transaction_results[[k]]$Buys$BuyPrice[j]))
  }
  cat(sprintf("Profit/Loss: %.2f\n", transaction_results[[k]]$PnL))
  cat("-------------------------------\n\n")
}

cat(sprintf("Transactions Summary: %d\n", k+1))
print(paste0("Stock Name: ", stock_id))
print(paste0("Shares Bought: ", total_shares_bought))
print(paste0("Shares Sold: ", total_shares_sold))
share_balance <- total_shares_bought - total_shares_sold
print(paste0("Shares Remaining: ", share_balance))
print(paste0("Total Costs from Buys: ", total_costs))
print(paste0("Total Proceeds from Sells: ", total_proceeds))    # s/b -527330.07  /  678529.47
g_l = c(total_proceeds - total_costs)
print(paste0("Total Gain/<loss> : ", g_l))
cat("End of Summary : ", k+1, "\n")


Transaction 1
Sell
2023-03-02  -50  21.375
Buys
2023-02-21  50  22.345
Profit/Loss: -48.50
-------------------------------

Transaction 2
Sell
2023-03-02  -1275  21.265
Buys
2023-02-21  750  22.345
2023-02-21  80  22.548
2023-02-21  445  21.748
Profit/Loss: -1127.57
-------------------------------

Transaction 3
Sell
2023-03-02  -215  21.271
Buys
2023-02-21  215  21.748
Profit/Loss: -102.56
-------------------------------

Transaction 4
Sell
2023-03-02  -510  21.270
Buys
2023-02-21  140  21.748
2023-03-01  370  20.695
Profit/Loss: 145.83
-------------------------------

Transaction 5
Sell
2023-03-03  -300  26.200
Buys
2023-03-01  300  20.695
Profit/Loss: 1651.50
-------------------------------

Transaction 6
Sell
2023-03-03  -70  26.201
Buys
2023-03-01  70  20.695
Profit/Loss: 385.42
-------------------------------

Transaction 7
Sell
2023-03-03  -600  26.202
Buys
2023-03-01  260  20.695
2023-03-01  340  20.880
Profit/Loss: 3241.30
-------------------------------

Transaction 8
Sell
20

In [6]:
total_shares_sold
total_shares_bought 

[1] 22080

[1] 22080

### <font color = yellow> Write to Disk - preserves the formatting of each Transaction


In [7]:
# Function to write transactions to disk with preserved formatting
write_transactions_to_disk <- function(transaction_results, file_name = "transactions.txt") {
  # Open a file connection
  con <- file(file_name, open = "wt")
  
  for (i in 1:length(transaction_results)) {
    transaction <- transaction_results[[i]]
    
    # Write the transaction details to the file
    writeLines(sprintf("Transaction %d", i), con)
    writeLines("Sell", con)
    writeLines(sprintf("%s\t%d\t%.3f", transaction$SellDate, transaction$SellQty, transaction$SellPrice), con)
    writeLines("Buys", con)
    
    for (j in 1:nrow(transaction$Buys)) {
      writeLines(sprintf("%s\t%d\t%.3f", transaction$Buys$BuyDate[j], transaction$Buys$BuyQty[j], transaction$Buys$BuyPrice[j]), con)
    }
    
    writeLines(sprintf("Profit/Loss: %.2f", transaction$PnL), con)
    writeLines("-------------------------------", con)
    writeLines("", con)  # Add a blank line for separation
  }
  
  # Close the file connection
  close(con)
}

# Call the function to write transactions to disk
write_transactions_to_disk(transaction_results)

#  Print summary info:
cat(sprintf("Transactions Summary: %d\n", k+1))
cat("Stock Name: ", stock_id, "\n")
cat("Total Buys: ", total_shares_bought, "\n")
cat("Total Sells: ", total_shares_sold, "\n")
cat("Shares Remaining: ", share_balance, "\n")
cat("Total Costs from Buys: ", total_costs, "\n")
cat("Total Proceeds from Sells: ", total_proceeds, "\n")
# g_l <- total_proceeds - total_costs
cat("Total Gain/<Loss>: ", g_l, "\n")
cat("End of Summary\n")


Transactions Summary: 32
Stock Name:  AI 
Total Buys:  22080 
Total Sells:  22080 
Shares Remaining:  0 
Total Costs from Buys:  527329.8 
Total Proceeds from Sells:  678538.9 
Total Gain/<Loss>:  151209.1 
End of Summary


### <font color = yellow> Append the Tranactions.txt w/ Summary Transaction information

In [8]:
# Function to append the summary to the existing file
append_summary_to_disk <- function(file_name = "transactions.txt") {
  # Open a file connection in append mode
  con <- file(file_name, open = "at")
  
  # Write the summary to the file
  writeLines(sprintf("Transactions Summary: %d", k+1), con)
  writeLines(sprintf("Stock ID: %s", stock_id ), con)  
    
  writeLines(sprintf("Total Buys: %d", total_shares_bought), con)
  writeLines(sprintf("Total Sells: %d", total_shares_sold), con)
  writeLines(sprintf("Remaining Shares: %d", share_balance ), con)    
    
  writeLines(sprintf("Total Costs from Buys: %.2f", total_costs), con)
  writeLines(sprintf("Total Proceeds from Sells: %.2f", total_proceeds), con)
  g_l <- total_proceeds - total_costs
  writeLines(sprintf("Total Gain/<Loss>: %.2f", g_l), con)
  writeLines("End of Summary", con)
  
  # Close the file connection
  close(con)
}

# Call the function to append the summary to the file
append_summary_to_disk()

# Display the summary info
cat(sprintf("Transactions Summary (append to disk file): %d\n", k+1))
cat("Stock Name: ", stock_id, "\n")
print(paste0("Shares Remaining: ", share_balance))
cat("Total Buys: ", total_shares_bought, "\n")
cat("Total Sells: ", total_shares_sold, "\n")
cat("Stock Remaining: ", share_balance, "\n")
cat("Total Costs from Buys: ", total_costs, "\n")
cat("Total Proceeds from Sells: ", total_proceeds, "\n")
g_l <- total_proceeds - total_costs
cat("Total Gain/<Loss>: ", g_l, "\n")
cat("End of Summary\n")


Transactions Summary (append to disk file): 32
Stock Name:  AI 
[1] "Shares Remaining: 0"
Total Buys:  22080 
Total Sells:  22080 
Stock Remaining:  0 
Total Costs from Buys:  527329.8 
Total Proceeds from Sells:  678538.9 
Total Gain/<Loss>:  151209.1 
End of Summary
